In [1]:
import requests

In [4]:
image_path = "../data/sampling/document_1/sample_0.png"
sample_id = "1"
document_id = "1"

In [8]:
url = "http://localhost:8000/document_generator/api/sample_documents/"
files = {
    "image": open(image_path, "rb")
}
data = {
    "name": f"sample_{sample_id}",
    "template_document": document_id
}

with requests.Session() as session:
    response = session.post(url, files=files, data=data)

In [9]:
response.json()

{'id': 3,
 'name': 'sample_1',
 'image': 'http://localhost:8000/media/document_samples/sample_0_WnvVe1D.png',
 'template_document': 1}